In [4]:
import os
from edinet_xbrl.edinet_xbrl_downloader import EdinetXbrlDownloader
import pandas as pd
import shutil

def download():

    xbrl_downloader = EdinetXbrlDownloader()
 #   xbrldir = 'C:/Users/shimizu/data/xbrl'
    xbrldir = 'D:/lab\edinet'
 
    # エクセルからtickerのリストを読み込む
    Listexcel = pd.read_excel('D:/lab/edinet/List_sell.xlsx')
    List = Listexcel['ticker']

    # 複数のTickerに対して処理を繰り返す---------------------------
    cnt = 0
    for tic in List:
        cnt = cnt +1
        ticker = tic
        print("cnt=",cnt)
        print('今は'+str(tic)+'の処理をしています')
 
        target_dir = os.path.join(xbrldir, str(ticker))
        if os.path.isdir(target_dir)==True:# dirがある場合
            xbrl_downloader.download_by_ticker(str(ticker), target_dir)# すでに存在するdirにダウンロードする
            print('フォルダがありました')
        else:# dirがない場合
            os.mkdir(os.path.join(xbrldir, str(ticker)))# dirを作成
            xbrl_downloader.download_by_ticker(str(ticker), target_dir)# 作成したdirにダウンロードする
            print('フォルダを新たに作成しました')
 
if __name__ == '__main__': #直接実行されたときのみ実行
    download()

In [11]:
# coding: utf-8

#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#    http://www.apache.org/licenses/LICENSE-2.0
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License. See accompanying LICENSE file.


class EdinetDataUtil(object):
    CONTEXT_REF = "contextref"
    UNIT_REF = "unitref"
    DECIMALS = "decimals"
    FORMAT = "format"
    NAME = "name"
    SCALE = "scale"

    @staticmethod
    def get_key(node):
        return node.name

    @staticmethod
    def get_value(node):
        return node.string

    @staticmethod
    def offset(num, power):
        return num * pow(10, power)

    @staticmethod
    def _get_ref_str_value(node, key):
        return node.attrs.get(key, "")

    @staticmethod
    def _get_ref_int_value(node, key):
        return int(node.attrs.get(key, 0))

    @classmethod
    def get_context_ref(cls, node):
        return cls._get_ref_str_value(node, cls.CONTEXT_REF)

    @classmethod
    def get_unit_ref(cls, node):
        return cls._get_ref_str_value(node, cls.UNIT_REF)

    @classmethod
    def get_decimals(cls, node):
        return cls._get_ref_int_value(node, cls.DECIMALS)

    @classmethod
    def get_format(cls, node):
        return cls._get_ref_str_value(node, cls.FORMAT)

    @classmethod
    def get_name(cls, node):
        return cls._get_ref_str_value(node, cls.NAME)

    @classmethod
    def get_scale(cls, node):
        return cls._get_ref_int_value(node, cls.SCALE)


In [12]:
# coding: utf-8

#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#    http://www.apache.org/licenses/LICENSE-2.0
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License. See accompanying LICENSE file.

import os
from edinet_xbrl.ufocatcher_util import UfoCatcherUtil
from time import sleep
import urllib.request


class EdinetXbrlDownloader(object):
    @classmethod
    def download(cls, url, target_dir, is_override, is_numbering):
        file_name = "{0}/{1}".format(target_dir, url.split("/")[-1])
        if os.path.exists(file_name):
            if not is_override:
                return
            if is_numbering:
                file_name = cls.__get_next_file_name(file_name)
        urllib.request.urlretrieve(url, file_name)

    @classmethod
    def download_by_ticker(cls, ticker, target_dir, wait_sec=1.0, type_of_document="",
                           is_override=True, is_numbering=False):
        response = UfoCatcherUtil.request(ticker)
        for url in UfoCatcherUtil.generate_edinet_xbrl_url(response.text):
            if not cls.__is_type_of_document(url, type_of_document):
                continue
            cls.download(url, target_dir, is_override, is_numbering)
            sleep(wait_sec)

    @staticmethod
    def __is_type_of_document(url, type_of_document):
        """
        Specify the Type of document of url
        If you don't want to specify the type of document -> type_of_document = ""
        """
        if type_of_document == "":
            return True
        return type_of_document in url.split('-')

    @staticmethod
    def __get_next_file_name(fn):
        """
        Add number for identify
        ex: '~(1)', '~(2)', ...
        """
        name, expanded = (lambda sfn: ("".join(sfn[:-1]), sfn[-1]))(fn.split('.'))
        c = 1
        while os.path.exists(name+"({})".format(c)+expanded):
            c += 1
        return name + "({})".format(c) + expanded



In [16]:
# coding: utf-8

#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#    http://www.apache.org/licenses/LICENSE-2.0
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License. See accompanying LICENSE file.

from collections import OrderedDict
from edinet_xbrl.edinet_data_util import EdinetDataUtil


class EdinetData(object):
    def __init__(self, key, value, decimals=0, unit_ref="", context_ref=""):
        self.key = key
        self.value = value
        self.decimals = decimals
        self.unit_ref = unit_ref
        self.context_ref = context_ref

    def get_key(self):
        return self.key

    def get_value(self):
        return self.value

    def get_decimals(self):
        return self.decimals

    def get_unit_ref(self):
        return self.unit_ref

    def get_context_ref(self):
        return self.context_ref

    @staticmethod
    def create(node):
        key = EdinetDataUtil.get_key(node)
        value = EdinetDataUtil.get_value(node)
        decimals = EdinetDataUtil.get_decimals(node)
        unit_ref = EdinetDataUtil.get_unit_ref(node)
        context_ref = EdinetDataUtil.get_context_ref(node)
        return EdinetData(key, value, decimals, unit_ref, context_ref)


class EdinetXbrlObject(object):
    def __init__(self):
        self._data_dict = OrderedDict()

    def clear(self):
        self._data_dict.clear()

    def put(self, key, edinet_data):
        if self.has_key(key):
            self._data_dict[key].append(edinet_data)
        else:
            self._data_dict[key] = [edinet_data]

    def get_data_list(self, key, auto_lower=True):
        if auto_lower:
            key = key.lower()

        return self._data_dict.get(key, [])

    def get_data_by_context_ref(self, key, context_ref):
        val = list(filter(lambda d: d.get_context_ref() == context_ref, self.get_data_list(key)))
        if val:
            return val[0]
        else:
            return None

    def get_keys(self):
        return self._data_dict.keys()

    def has_key(self, key):
        return key in self._data_dict


In [20]:
# coding: utf-8

#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#    http://www.apache.org/licenses/LICENSE-2.0
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License. See accompanying LICENSE file.

from xbrl import XBRLParser
from edinet_xbrl.edinet_xbrl_object import EdinetData, EdinetXbrlObject
import codecs


class EdinetXbrlParser(object):
    @classmethod
    def parse_file(cls, xbrl_file_path):
        data_container = EdinetXbrlObject()
        with codecs.open(xbrl_file_path, 'r', 'utf-8') as of:
            parser = XBRLParser.parse(of)
            for node in parser.find_all():
                cls.put_node(data_container, node)
        return data_container

    @staticmethod
    def put_node(data_container, node):
        data_container.put(node.name, EdinetData.create(node))


In [18]:
# coding: utf-8

#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#    http://www.apache.org/licenses/LICENSE-2.0
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License. See accompanying LICENSE file.

import xml.etree.ElementTree as ET
import requests


class UfoCatcherUtil(object):
    NAMESPACE = '{http://www.w3.org/2005/Atom}'
    FAKE_HTTP_HEADER = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36))"}
    UFOCATCH_URL = 'http://resource.ufocatch.com/atom/edinetx/query'

    @classmethod
    def create_url(cls, query):
        return "{0}/{1}".format(cls.UFOCATCH_URL, query)

    @classmethod
    def request(cls, tikcer, headers=FAKE_HTTP_HEADER):
        return requests.get(url=cls.create_url(tikcer), headers=headers)

    @classmethod
    def is_edinet_xbrl_url(cls, element):
        url = cls.get_href_attrib(element)
        return False if url is None else ("PublicDoc" in url and url.endswith(".xbrl"))

    @classmethod
    def get_et_tree(cls, html):
        et_tree = ET.fromstring(html)
        ET.register_namespace('', cls.NAMESPACE[1:-1])
        return et_tree

    @classmethod
    def generate_edinet_xbrl_url(cls, html):
        et_tree = cls.get_et_tree(html)
        for el in et_tree.findall('.//' + cls.NAMESPACE + 'entry'):
            for link in el.findall('./' + cls.NAMESPACE + 'link[@type="text/xml"]'):
                if cls.is_edinet_xbrl_url(link):
                    yield cls.get_href_attrib(link)

    @staticmethod
    def get_href_attrib(element):
        return element.attrib.get('href', None)


In [6]:
# coding: utf-8

#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#    http://www.apache.org/licenses/LICENSE-2.0
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License. See accompanying LICENSE file.


import os
import unittest
from nose.tools import eq_
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
import yaml
import codecs


class EdinetXbrlParserTestCase(unittest.TestCase):
    TEST_DIR = "{0}/test_data".format(os.path.dirname(os.path.abspath(__file__)))
    EMPLOYEES_NUM_KEY = "jpcrp_cor:NumberOfEmployees"
    ASSETS_NUM_KEY = "jppfs_cor:Assets"
    NETSALES_KEY = "jppfs_cor:NetSales"
    CURRENT_YEAR_INSTANT_CONTEXT = "CurrentYearInstant"
    CURRENT_YEAR_DURATION_CONTEXT = "CurrentYearDuration"
    CURRENT_YEAR_INSTANT_NON_CON_CONTEXT = "CurrentYearInstant_NonConsolidatedMember"
    CURRENT_YEAR_DURATION_NON_CON_CONTEXT = "CurrentYearDuration_NonConsolidatedMember"

    @staticmethod
    def parse(xbrl_file_path):
        parser = EdinetXbrlParser()
        return parser.parse_file(xbrl_file_path)

    @classmethod
    def get_xbrl_file(cls, target):
        return "{0}/{1}.xbrl".format(cls.TEST_DIR, target)

    @classmethod
    def get_expected_dict(cls, target):
        yaml_file = "{0}/{1}.yaml".format(cls.TEST_DIR, target)
        with codecs.open(yaml_file, 'r', 'utf-8') as f:
            return yaml.load(f)

    @classmethod
    def test_kakaku(cls):
        target = "CJ_2371_kakakucom"
        xbrl_file = cls.get_xbrl_file(target)
        data_container = cls.parse(xbrl_file)
        expected_dict = cls.get_expected_dict(target)
        eq_(expected_dict["employees_num"],
            int(data_container.get_data_by_context_ref(cls.EMPLOYEES_NUM_KEY,
                                                       cls.CURRENT_YEAR_INSTANT_CONTEXT).get_value()))
        eq_(expected_dict["assets"],
            int(data_container.get_data_by_context_ref(cls.ASSETS_NUM_KEY,
                                                       cls.CURRENT_YEAR_INSTANT_CONTEXT).get_value()))
        eq_(expected_dict["netsales"],
            int(data_container.get_data_by_context_ref(cls.NETSALES_KEY,
                                                       cls.CURRENT_YEAR_DURATION_CONTEXT).get_value()))

    @classmethod
    def test_yahoo(cls):
        target = "CI_4689_yahoo"
        xbrl_file = cls.get_xbrl_file(target)
        data_container = cls.parse(xbrl_file)
        expected_dict = cls.get_expected_dict(target)
        eq_(expected_dict["employees_num"],
            int(data_container.get_data_by_context_ref(cls.EMPLOYEES_NUM_KEY,
                                                       cls.CURRENT_YEAR_INSTANT_NON_CON_CONTEXT).get_value()))
        eq_(expected_dict["assets"],
            int(data_container.get_data_by_context_ref(cls.ASSETS_NUM_KEY,
                                                       cls.CURRENT_YEAR_INSTANT_NON_CON_CONTEXT).get_value()))
        eq_(expected_dict["netsales"],
            int(data_container.get_data_by_context_ref(cls.NETSALES_KEY,
                                                       cls.CURRENT_YEAR_DURATION_NON_CON_CONTEXT).get_value()))


if __name__ == '__main__':
    unittest.main()


NameError: name '__file__' is not defined

In [14]:
# coding: utf-8

#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#    http://www.apache.org/licenses/LICENSE-2.0
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License. See accompanying LICENSE file.


import unittest
from xml.etree.ElementTree import Element
from nose.tools import eq_, assert_true, assert_false
from edinet_xbrl.ufocatcher_util import UfoCatcherUtil


class UfoCatcherUtilTest(unittest.TestCase):
    @staticmethod
    def test_create_url():
        query = 'aaa'
        eq_('http://resource.ufocatch.com/atom/edinetx/query/' + query, UfoCatcherUtil.create_url(query))

    @staticmethod
    def test_is_edinet_xbrl_url():
        # true
        ok_element = Element("", {"href": "http://aaa.bbb/PublicDoc/ccc.xbrl"})
        assert_true(UfoCatcherUtil.is_edinet_xbrl_url(ok_element))

        # false
        for ng_element in [Element("", {"href": "http://aaa.bbb/ccc.xbrl"}),
                           Element("", {"href": "http://aaa.bbb/PublicDoc/ccc.html"})]:
            assert_false(UfoCatcherUtil.is_edinet_xbrl_url(ng_element))

    @staticmethod
    def test_get_href_attrib():
        href = "http://aaa.bbb/PublicDoc/ccc.xbrl"
        element = Element("", {"href": href})
        eq_(href, UfoCatcherUtil.get_href_attrib(element))
        # if href attrib is unset
        eq_(None, UfoCatcherUtil.get_href_attrib(Element("")))


if __name__ == '__main__':
    unittest.main()


E
ERROR: C:\Users\yuu-pc\AppData\Roaming\jupyter\runtime\kernel-99433558-d195-47ac-98ca-af40e99fba87 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\yuu-pc\AppData\Roaming\jupyter\runtime\kernel-99433558-d195-47ac-98ca-af40e99fba87'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
"""A setuptools based setup module.

See:
https://packaging.python.org/en/latest/distributing.html
https://github.com/pypa/sampleproject
"""

# Always prefer setuptools over distutils
from setuptools import setup, find_packages
# To use a consistent encoding
from codecs import open
from os import path

here = path.abspath(path.dirname(__file__))

# Get the long description from the README file
with open(path.join(here, 'README.rst'), encoding='utf-8') as f:
    long_description = f.read()

# Arguments marked as "Required" below must be included for upload to PyPI.
# Fields marked as "Optional" may be commented out.

setup(
    # This is the name of your project. The first time you publish this
    # package, this name will be registered for you. It will determine how
    # users can install this project, e.g.:
    #
    # $ pip install sampleproject
    #
    # And where it will live on PyPI: https://pypi.org/project/sampleproject/
    #
    # There are some restrictions on what makes a valid project name
    # specification here:
    # https://packaging.python.org/specifications/core-metadata/#name
    name='edinet_xbrl',  # Required

    # Versions should comply with PEP 440:
    # https://www.python.org/dev/peps/pep-0440/
    #
    # For a discussion on single-sourcing the version across setup.py and the
    # project code, see
    # https://packaging.python.org/en/latest/single_source_version.html
    version='0.2.0',  # Required

    # This is a one-line description or tagline of what your project does. This
    # corresponds to the "Summary" metadata field:
    # https://packaging.python.org/specifications/core-metadata/#summary
    description='A Python Edinet xbrl file parser',  # Required

    # This is an optional longer description of your project that represents
    # the body of text which users will see when they visit PyPI.
    #
    # Often, this is the same as your README, so you can just read it in from
    # that file directly (as we have already done above)
    #
    # This field corresponds to the "Description" metadata field:
    # https://packaging.python.org/specifications/core-metadata/#description-optional
    long_description=long_description,  # Optional

    # This should be a valid link to your project's main homepage.
    #
    # This field corresponds to the "Home-Page" metadata field:
    # https://packaging.python.org/specifications/core-metadata/#home-page-optional
    url='https://github.com/BuffetCode/edinet_xbrl',  # Optional

    # This should be your name or the name of the organization which owns the
    # project.
    author='https://github.com/shoe116',  # Optional

    # This should be a valid email address corresponding to the author listed
    # above.
    author_email='buffett.code.info@gmail.com',  # Optional

    # Pick your license as you wish
    license='Apache License 2.0',

    # Classifiers help users find your project by categorizing it.
    #
    # For a list of valid classifiers, see
    # https://pypi.python.org/pypi?%3Aaction=list_classifiers
    classifiers=[  # Optional
        # How mature is this project? Common values are
        #   3 - Alpha
        #   4 - Beta
        #   5 - Production/Stable
        'Development Status :: 4 - Beta',

        # Indicate who your project is intended for
        'Intended Audience :: Developers',
        'Topic :: Software Development :: Build Tools',

        # Specify the Python versions you support here. In particular, ensure
        # that you indicate whether you support Python 2, Python 3 or both.
        'Programming Language :: Python :: 3.3',
        'Programming Language :: Python :: 3.4',
        'Programming Language :: Python :: 3.5',
        'Programming Language :: Python :: 3.6',
    ],

    # This field adds keywords for your project which will appear on the
    # project page. What does your project relate to?
    #
    # Note that this is a string of words separated by whitespace, not a list.
    keywords='edinet xbrl parser for python',  # Optional

    # You can just specify package directories manually here if your project is
    # simple. Or you can use find_packages().
    #
    # Alternatively, if you just want to distribute a single Python file, use
    # the `py_modules` argument instead as follows, which will expect a file
    # called `my_module.py` to exist:
    #
    #   py_modules=["my_module"],
    #
    packages=find_packages(exclude=['contrib', 'docs', 'tests']),  # Required

    # This field lists other packages that your project depends on to run.
    # Any package you put here will be installed by pip when your project is
    # installed, so they must be valid existing projects.
    #
    # For an analysis of "install_requires" vs pip's requirements files see:
    # https://packaging.python.org/en/latest/requirements.html
    install_requires=['python-xbrl', 'requests'],  # Optional

    # List additional groups of dependencies here (e.g. development
    # dependencies). Users will be able to install these using the "extras"
    # syntax, for example:
    #
    #   $ pip install sampleproject[dev]
    #
    # Similar to `install_requires` above, these must be valid existing
    # projects.
    extras_require={  # Optional
        'dev': ['nose', 'pyyaml', 'requests'],
        'test': ['nose', 'pyyaml', 'requests'],
    },

    # If there are data files included in your packages that need to be
    # installed, specify them here.
    #
    # If using Python 2.6 or earlier, then these have to be included in
    # MANIFEST.in as well.
    package_data={  # Optional
    },

    # Although 'package_data' is the preferred approach, in some case you may
    # need to place data files outside of your packages. See:
    # http://docs.python.org/3.4/distutils/setupscript.html#installing-additional-files
    #
    # In this case, 'data_file' will be installed into '<sys.prefix>/my_data'
    data_files=[],  # Optional

    # To provide executable scripts, use entry points in preference to the
    # "scripts" keyword. Entry points provide cross-platform support and allow
    # `pip` to create the appropriate form of executable for the target
    # platform.
    #
    # For example, the following would provide a command called `sample` which
    # executes the function `main` from this package when invoked:
    entry_points={  # Optional
    },

    # To provide test directly
    test_suite='tests'

)


NameError: name '__file__' is not defined

In [10]:
import os

print("__file__: %r" %__file__)
print("os.path.dirname(__file__)                 : %r" % (os.path.dirname(__file__)))
print("os.path.abspath(__file__)                 : %r" % (os.path.abspath(__file__)))
print("os.path.dirname(os.path.abspath(__file__)): %r" % (os.path.dirname(os.path.abspath(__file__)))

NameError: name '__file__' is not defined

In [14]:
import os


current_dir1 = os.path.dirname(os.path.abspath("__file__"))

current_dir2 = os.path.dirname(os.path.abspath('__file__'))

current_dir3 = os.path.dirname(os.path.abspath(sys.argv[0]))

print(current_dir1)
print(current_dir2)
print(current_dir3)

NameError: name 'sys' is not defined

In [13]:
import os
dir_path = pathlib.Path(__file__).parent.resolve()

NameError: name 'pathlib' is not defined

In [15]:
import os
print("os.path.dirname(os.path.abspath('__file__')): %r" % (os.path.dirname(os.path.abspath('__file__'))))
print("os.path.dirname(os.path.abspath(__file__)): %r" % (os.path.dirname(os.path.abspath(__file__))))

os.path.dirname(os.path.abspath('__file__')): 'D:\\python\\program'


NameError: name '__file__' is not defined

In [16]:
import os
os.path.dirname(os.path.abspath("__file__"))

'D:\\python\\program'

In [17]:
os.path.dirname(os.path.abspath("__file__"))

'D:\\python\\program'

In [18]:
import os

print("os.path.dirname(os.path.abspath('__file__')): %r" % (os.path.dirname(os.path.abspath('__file__'))))

os.path.dirname(os.path.abspath('__file__')): 'D:\\python\\program'


In [19]:
import os

print("os.path.dirname(os.path.abspath('__file__')): %r" % (os.path.dirname(os.path.abspath('__file__'))))
print("os.path.dirname(os.path.abspath('__file__')): %s" % (os.path.dirname(os.path.abspath('__file__'))))

os.path.dirname(os.path.abspath('__file__')): 'D:\\python\\program'
os.path.dirname(os.path.abspath('__file__')): D:\python\program


In [20]:
from pathlib import Path

print("Path().resolve('__file__'): %r" % (Path().resolve('__file__')))
print("Path().resolve(): %r" % (Path().resolve()))
print("Path().resolve(): %s" % (Path().resolve()))

Path().resolve('__file__'): WindowsPath('D:/python/program')
Path().resolve(): WindowsPath('D:/python/program')
Path().resolve(): D:\python\program
